In [25]:
import sys
import os
sys.path.append(os.path.abspath("../src"))


### Example 1: Fluid Circuit Building Basics
---

The below code builds and plots a simple fluid circuit using the `element` base class. In practice, subclasses of `element` like `pipe` or `orifice` are used to build a physical circuit model, as in the following exapmle, but this demonstrates the basic functionality.

In [19]:
from network import network
from network import element

# Instantiate test elements using base class
component1 = element("COMP-1", 3)
component2 = element("COMP-2", 2)
component3 = element("COMP-3", 2)
component4 = element("COMP-4", 3)

# Connect desired ports to eachother
component1.tie_in(component2, 1, 0)
component1.tie_in(component3, 2, 0)
component2.tie_in(component4, 1, 0)
component3.tie_in(component4, 1, 1)

# Create network (meshing)
circuit1 = network(component1)

# Run qualitative checks
circuit1.mesh_checks()


Bulk nodes:  [1, 2, 3, 4]
Boundary nodes:  [0, 5]

Nodes by element: 
-----------------
COMP-1: [0, 1, 2]
COMP-2: [1, 3]
COMP-3: [2, 4]
COMP-4: [3, 4, 5]

Neighbors by element: 
---------------------
COMP-1:
   Port 0: Boundary
   Port 1: COMP-2
   Port 2: COMP-3
COMP-2:
   Port 0: COMP-1
   Port 1: COMP-4
COMP-3:
   Port 0: COMP-1
   Port 1: COMP-4
COMP-4:
   Port 0: COMP-2
   Port 1: COMP-3
   Port 2: Boundary


### Example 2: Simple Steady Flow Problem
---
Lets take a simple, now physical, circuit:
![image](simple_circuit_1.png)

If a fluid enters from the left, it will diverge, flow through the two branches and reconverge before exiting on the right. In the top branch, the flow will be restricted by the orifice. In the bottom branch, the two reducers will also restrict the flow. Then, we can discretize the circuit into a handful of pipes, manifolds, reducers, and an orifice, all connected at their various ports:

![image](simple_circuit_1_annotated.png)

Modeling this, as we did in example 1:

In [29]:
import pipe
import manifold as man
import orifice as ofc
import reducer as red
import network as net

# Stupid re-import issue
import importlib
importlib.reload(pipe)
importlib.reload(manifold)
importlib.reload(orifice)
importlib.reload(reducer)
importlib.reload(network)


### Defining pipe objects
### ---------------------
Dp = 1*.0254# [m] typical pipe diameter
Dp4 = .5*.0254# [m] pipe 4 diameter
Lp = .25 # [m] typical pipe length
Lp25 = .5 # [m] pipes 2 and 5 length
roughness = .001 # [m] typical pipe roughness
r_bend = 2*.0254 # [m] typical pipe bend radius
a_bend = 90 # [deg] typical pipe bend angle

pipe1 = pipe.pipe("PIPE-1", Lp, Dp, roughness)
pipe2 = pipe.pipe("PIPE-2", Lp25, Dp, roughness, r_bend, a_bend)
pipe3 = pipe.pipe("PIPE-3", Lp, Dp, roughness, r_bend, a_bend)
pipe4 = pipe.pipe("PIPE-4", Lp, Dp4, roughness)
pipe5 = pipe.pipe("PIPE-5", Lp25, Dp, roughness, r_bend, a_bend)
pipe6 = pipe.pipe("PIPE-6", Lp, Dp, roughness, r_bend, a_bend)
pipe7 = pipe.pipe("PIPE-7", Lp, Dp, roughness)
             
    
### Defining manifold objects
### -------------------------
man_ports = 3 # typical number of ports in manifold

manifold1 = man.manifold("MAN-1", man_ports)
manifold2 = man.manifold("MAN-2", man_ports)
          
    
### Defining orifice object
### -----------------------
Do = .25*.0254 # [m] orifice diameter

orifice1 = ofc.orifice("OFC-1", Do)


### Defining reducer objects
### ------------------------
a_red = 30 # [deg] typical reducing angle
L_red = 2*.0254 # [m] typical reducer length

reducer1 = red.reducer("RED-1", Dp, Dp4, L_red, a_red, roughness)
reducer2 = red.reducer("RED-2", Dp, Dp4, L_red, a_red, roughness)


### Building the network
### --------------------
manifold1.tie_in(pipe1, 0, 1)
manifold1.tie_in(pipe2, 1, 0)
manifold1.tie_in(pipe3, 2, 0)

orifice1.tie_in(pipe2, 0, 1)
orifice1.tie_in(pipe5, 1, 0)

reducer1.tie_in(pipe3, 0, 1)
reducer1.tie_in(pipe4, 1, 0)
reducer2.tie_in(pipe4, 1, 1)
reducer2.tie_in(pipe6, 0, 0)

manifold2.tie_in(pipe5, 0, 1)
manifold2.tie_in(pipe6, 1, 1)
manifold2.tie_in(pipe7, 2, 0)

circuit2 = net.network(pipe1)
circuit2.mesh_checks()


Bulk nodes:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
Boundary nodes:  [0, 13]

Nodes by element: 
-----------------
PIPE-1: [0, 1]
MAN-1: [1, 2, 3]
PIPE-2: [2, 4]
PIPE-3: [3, 5]
OFC-1: [4, 6]
RED-1: [5, 7]
PIPE-5: [6, 8]
PIPE-4: [7, 9]
MAN-2: [8, 10, 11]
RED-2: [12, 9]
PIPE-6: [12, 10]
PIPE-7: [11, 13]

Neighbors by element: 
---------------------
PIPE-1:
   Port 0: Boundary
   Port 1: MAN-1
MAN-1:
   Port 0: PIPE-1
   Port 1: PIPE-2
   Port 2: PIPE-3
PIPE-2:
   Port 0: MAN-1
   Port 1: OFC-1
PIPE-3:
   Port 0: MAN-1
   Port 1: RED-1
OFC-1:
   Port 0: PIPE-2
   Port 1: PIPE-5
RED-1:
   Port 0: PIPE-3
   Port 1: PIPE-4
PIPE-5:
   Port 0: OFC-1
   Port 1: MAN-2
PIPE-4:
   Port 0: RED-1
   Port 1: RED-2
MAN-2:
   Port 0: PIPE-5
   Port 1: PIPE-6
   Port 2: PIPE-7
RED-2:
   Port 0: PIPE-6
   Port 1: PIPE-4
PIPE-6:
   Port 0: RED-2
   Port 1: MAN-2
PIPE-7:
   Port 0: MAN-2
   Port 1: Boundary
